In [2]:
from __future__ import print_function
import os
import numpy as np
import cv2
import pywt as py
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten, AveragePooling2D, GlobalMaxPooling2D
#from keras.layers import Dense, Conv3D, MaxPooling3D, Dropout, Flatten, AveragePooling3D
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU
from keras.utils import to_categorical

ImportError: No module named cv2

In [ ]:
ca = ''
co = ''
#path to database ca contains path to case data and co contains path to control data
NUM_CLASSES = 2 # The number of classes
IMG_SIZE = 1024 # The length of one side of images
COLOR_CHANNELS = 3 # RGB
input_shape = (IMG_SIZE, IMG_SIZE)

In [ ]:
train_image = []
train_label = []
train_img_dirs = [case, control]
for i, d in enumerate(train_img_dirs):
    files = os.listdir(d)
    for f in files:
        img = cv2.imread(d + '/'+ f)
        coeffs1 = py.dwt2(img, 'db3')#Here we are decomposing using 3-D wavelet. db3 wavelet is used here but one can change
        #as per requirment.
        LL, (LH, HL, HH) = coeffs1#storing co-efficients of wavelets
        img = HL[:,:,3]# Extracting only the component of wavelet which best extracts the region of interest. It can
        #be tested for other also like img = HL[:,:,1] or img = LH[:,:,3]. Best way is to see these using imshow.
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))#resizing the extracted image.
        img = img.flatten().astype(np.float32)/255.0
        train_image.append(img)
        tmp = np.zeros(NUM_CLASSES)
        tmp[i] = 1
        train_label.append(tmp)

In [ ]:
train_image = np.asarray(train_image)
train_label = np.asarray(train_label)
train_image = train_image.reshape(-1,IMG_SIZE, IMG_SIZE,1)
#Defining the array for training and label

In [ ]:
classes = np.unique(train_label)
nClasses = len(classes)
train_Y_one_hot = to_categorical(train_label)

In [ ]:
batch_size = 2
epochs = 25
num_classes = 2
#Defining batch size and no. of epochs

In [ ]:
#Defining the artitechture of CNN.The number of layers,activation function can be changed.
fashion_model = Sequential()
fashion_model.add(Conv2D(8, kernel_size=(3, 3),activation='linear',input_shape=(IMG_SIZE,IMG_SIZE,1),padding='same'))
fashion_model.add(LeakyReLU(alpha=0.1))#Relu activation function with alpha defined.Can be changed.
fashion_model.add(MaxPooling2D((2, 2),padding='same'))#maxpooling the convolved layer.Can be changed.
fashion_model.add(Conv2D(16, (3, 3), activation='linear',padding='same'))
fashion_model.add(LeakyReLU(alpha=0.1))
fashion_model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))
fashion_model.add(Conv2D(32, (3, 3), activation='linear',padding='same'))
fashion_model.add(LeakyReLU(alpha=0.1))
fashion_model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))
fashion_model.add(Conv2D(64, (3, 3), activation='linear',padding='same'))
fashion_model.add(LeakyReLU(alpha=0.1))
fashion_model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))
fashion_model.add(Flatten())
fashion_model.add(Dense(64, activation='linear'))
fashion_model.add(LeakyReLU(alpha=0.1))
fashion_model.add(Dropout(0.1))#adding dropout to remove layers with less dominance.
fashion_model.add(Dense(num_classes, activation='softmax'))#last layer to decied the binary classification.

In [ ]:
#defining otimization criteria of the model.
fashion_model.compile(loss=keras.losses.categorical_crossentropy,optimizer=keras.optimizers.Adam(),metrics=['accuracy'])
fashion_model.summary()

In [ ]:
fashion_train = fashion_model.fit(train_image, train_label, batch_size=batch_size,epochs=epochs,verbose=1)
#final fitting of the model.

In [ ]:
#testing the model on a dataset.
#path to be provided and testing dataset has to go through the same preprocessing
im0 = '' #provide path to test data set
image0 = []
img0_dirs = [im0]
for i , d in enumerate(img0_dirs):
    files = os.listdir(d)
    for f in files:
        img0 = cv2.imread(d + '/'+ f)
        #img0 = rgb2gray(img0)
        coeffs1 = py.dwt2(img0, 'db3')
        LL, (LH, HL, HH) = coeffs1
        img0 = HL[:,:,3]
        img0 = cv2.resize(img0, (IMG_SIZE, IMG_SIZE))
        img0 = img0.flatten().astype(np.float32)/255.0
        image0.append(img0)
        
test_image0 = np.asarray(image0)
test_image0 = test_image0.reshape(-1,IMG_SIZE, IMG_SIZE, 1)
test_eval = fashion_model.predict(test_image0, batch_size=None, verbose=0, steps=None)
print(np.rint(test_eval))